## Numba tutorial
It's a JIT on llvm (Backend part) that compile CPU/GPU code infered from python code. It needs less changes compared to cython, but needs that you fix the data type of the functions that you want to accelerate. To use is quite simple you just need to add a decorator(PRAGMA) to the python code.
In other words, Numba turns python into a compiled language with GPU/CPU target.

### Numba modes
* Object mode: Compiled code operates on python objects. Not fast only improve loop performance
* nopython mode: Full compiled code that operates on "machine native data"

### References
* https://eng.climate.com/2015/04/09/numba-vs-cython-how-to-choose/
* https://www.youtube.com/watch?v=QpaapVaL8Fw
* https://www.youtube.com/watch?v=eYIPEDnp5C4
* https://www.youtube.com/watch?v=06VErVj9MaQ&t=1509s
* https://www.youtube.com/watch?v=SzBi3xdEF2Y&t=4872s
* https://ipython.org/ipython-doc/3/interactive/magics.html
* http://jakevdp.github.io/blog/2012/08/24/numba-vs-cython/
* https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/
* http://numba.pydata.org/numba-doc/dev/user/examples.html
* https://julien.danjou.info/blog/2015/guide-to-python-profiling-cprofile-concrete-case-carbonara
* http://rlhick.people.wm.edu/posts/comparing-the-speed-of-matlab-versus-pythonnumpy.html
* http://stackoverflow.com/questions/5217167/how-many-cuda-cores-does-each-multiprocessor-of-a-gpu-have
* https://www.nvidia.com/en-us/geforce/products/10series/titan-x-pascal/
* https://docs.python.org/3.0/library/timeit.html
* http://numba.pydata.org/numba-doc/dev/user/examples.html
* http://numba.pydata.org/numba-doc/dev/user/vectorize.html
* http://numba.pydata.org/numba-doc/0.13/CUDAJit.html#
* http://numba.pydata.org/numba-doc/dev/cuda/index.html
* http://numba.pydata.org/numba-doc/dev/cuda/simulator.html
* http://python-3-patterns-idioms-test.readthedocs.io/en/latest/index.html
* https://blog.dominodatalab.com/interactive-dashboards-in-jupyter/

In [6]:
import numpy as np
from numba import jit

# Add jit decorator to use LLVM to compile to native code.
@jit
def calculate_mean_numba(x_vec):
    total = 0
    # Iterate on x_vec
    for xVal in x_vec:
        total = total + xVal
    
    return total / len(x_vec)

# Normal python
def calculate_mean(x_vec):
    total = 0
    # Iterate on x_vec
    for xVal in x_vec:
        total = total + xVal
    
    return total / len(x_vec)

# Using numpy
def calculate_mean_numpy(x_vec):
    return np.mean(x_vec)

In [7]:
# Create random vector with 10000 elements
big_vec = np.random.rand(10000)
print('Big vector shape:',big_vec.shape)

Big vector shape: (10000,)


### Using python version
Actually the simple naive for-loop on python is 10x slower than matlab
```matlab
function calcMean = simpleMean(vecIn)
	total = 0;
	for i=1:size(vecIn,1)
		total = total + vecIn(i);
	end
	calcMean = total / size(vecIn,1);
end

>> a = rand(10000,1);                
>> res1 = simpleMean(a)              

res1 =

    0.4996

>> res2 = mean(a)

res2 =

    0.4996

>> f = @() simpleMean(a);
>> timeit(f)                         

ans =

   1.5744e-04
```

In [8]:
%timeit calculate_mean(big_vec)
mean = calculate_mean(big_vec)
print('Mean value(Pure python) is %f' % (mean))

1000 loops, best of 3: 1.3 ms per loop
Mean value(Pure python) is 0.499933


In [9]:
%timeit calculate_mean_numpy(big_vec)
mean_numpy = calculate_mean_numpy(big_vec)
print('Mean value(Numpy) is %f' % (mean_numpy))

The slowest run took 6.62 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 18.7 µs per loop
Mean value(Numpy) is 0.499933


### Using Numba version

In [10]:
%timeit calculate_mean_numba(big_vec)
mean2 = calculate_mean_numba(big_vec)
print('Mean value(Numba CPU) is %f' % (mean2))

The slowest run took 6116.44 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 9.22 µs per loop
Mean value(Numba CPU) is 0.499933


## Using GPU
Numba also provide computations with CUDA, one of it's coolest features is that it accept numpy arrays. 
 * ufunc: Universal functions, element-wise functions.
 
 By using numba.vectorize you transform your scalar function into a element-wise function.

In [29]:
import numba.cuda
import math

In [90]:
myGpu = numba.cuda.get_current_device()
nMultiProcessors = myGpu.MULTIPROCESSOR_COUNT
# Check NVIDIA Architecture
nCoresPerCapability = {
    1:8,
    2:32,
    3:192,
    5:128,
    6:128
}
print ("Running on GPU", myGpu.name, "compute capability(major):", myGpu.compute_capability[0])
print ("Number of streaming multiprocessors:",nMultiProcessors)
print ("Number cores per multiprocessor:",nCoresPerCapability[myGpu.compute_capability[0]])
print ("Total cores on GPU:",nMultiProcessors*nCoresPerCapability[myGpu.compute_capability[0]])
print ("Warp size:", myGpu.WARP_SIZE)

Running on GPU b'TITAN X (Pascal)' compute capability(major): 6
Number of streaming multiprocessors: 28
Number cores per multiprocessor: 128
Total cores on GPU: 3584
Warp size: 32


### Vectorizes functions
Those functions will run on each element of the array

In [75]:
@numba.vectorize(['float32(float32)'], target='cpu')
def squareStuff_cpu(x):
    return x*x

@numba.vectorize(['float32(float32)'], target='parallel')
def squareStuff_parallel(x):
    return x*x

@numba.vectorize(['float32(float32)'], target='cuda')
def squareStuff_cuda(x):
    return x*x

@numba.vectorize(['float64(float64, float64)'], target='cpu')
def sin_cos_cpu(x,y):
    return math.sin(x)*math.cos(y)

@numba.vectorize(['float64(float64, float64)'], target='parallel')
def sin_cos_parallel(x,y):
    return math.sin(x)*math.cos(y)

@numba.vectorize(['float32(float64, float64)'], target='cuda')
def sin_cos_cuda(x,y):
    return math.sin(x)*math.cos(y)


In [71]:
# Create array from 0 to 19
a = np.arange(20, dtype=np.float32)

# Run on CPU
resCPU = squareStuff_cpu(a)

# Run on available cores
resPar = squareStuff_parallel(a)

# Run on available cores
resGPU = squareStuff_cuda(a)

# Run with timeit
print('CPU time')
%timeit squareStuff_cpu(a)
print('\nCPU cores time')
%timeit squareStuff_parallel(a)
print('\nGPU cores time')
%timeit squareStuff_cuda(a)

/usr/local/anaconda3/lib/python3.5/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


CPU time
The slowest run took 11.16 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 858 ns per loop

CPU cores time
1000 loops, best of 3: 219 µs per loop

GPU cores time


/usr/local/anaconda3/lib/python3.5/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


1000 loops, best of 3: 797 µs per loop


In [72]:
print('Result on CPU\n',resCPU)
print('Result on Cores\n',resPar)
print('Result on Gpu\n',resGPU)

Result on CPU
 [ 0.          0.84147096  0.98935825  0.95637596  0.92002606 -0.61604047
  0.69605851 -0.53659838  0.0795185   0.14993681  0.82687956 -0.86000788
  0.12372269 -0.85561359 -0.98363125  0.80131495 -0.59464198 -0.43578497
  0.93349361 -0.78533506]
Result on Cores
 [ 0.          0.84147096  0.98935825  0.95637596  0.92002606 -0.61604047
  0.69605851 -0.53659838  0.0795185   0.14993681  0.82687956 -0.86000788
  0.12372269 -0.85561359 -0.98363125  0.80131495 -0.59464198 -0.43578497
  0.93349361 -0.78533506]
Result on Gpu
 [ 0.          0.84147102  0.98935825  0.95637596  0.92002606 -0.61604047
  0.69605851 -0.53659838  0.07951849  0.14993681  0.82687956 -0.86000788
  0.12372269 -0.85561359 -0.98363125  0.80131495 -0.59464198 -0.43578494
  0.93349361 -0.78533512]


In [85]:
# Generate data
n = 80000000
x = np.linspace(0,np.pi,n)
y = np.linspace(0,np.pi,n)

# Check result
refAns = np.sin(x) * np.cos(y)
resCpuVec = sin_cos_cpu(x,y)
resCpuParVec = sin_cos_parallel(x,y)
resGpuVec = sin_cos_cuda(x,y)

#%timeit sin_cos_cuda(x,y)

print("CPU vectorize correct:", np.allclose(refAns,resCpuVec))
print("GPU vectorize correct:", np.allclose(refAns,resGpuVec))
print("CPU(par) vectorize correct:", np.allclose(refAns,resCpuParVec))

/usr/local/anaconda3/lib/python3.5/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


CPU vectorize correct: True
GPU vectorize correct: True
CPU(par) vectorize correct: True


In [86]:
# Run with timeit
print('CPU time')
%timeit sin_cos_cpu(x,y)
print('\nCPU cores time')
%timeit sin_cos_parallel(x,y)
print('\nGPU cores time')
%timeit sin_cos_cuda(x,y)

CPU time
1 loop, best of 3: 4.35 s per loop

CPU cores time
1 loop, best of 3: 594 ms per loop

GPU cores time


/usr/local/anaconda3/lib/python3.5/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


1 loop, best of 3: 938 ms per loop


### Creating Cuda cores on python
Numba support the creation of cuda kernels using python

#### Create Kernel

In [133]:
from numba import cuda

# Create a kernel that add 2 vectors
@numba.cuda.jit('void(float32[:], float32[:], float32[:])')
def addVecsKernel(vec_a, vec_b, vec_out):
    
    # Basic code (almost all kernels do the same) to get the kernel index coordinates
    thread_x = cuda.threadIdx.x
    block_x = cuda.blockIdx.x
    threads_per_block = cuda.blockDim.x
    
    # Calculate thread index
    idxThread = thread_x+ block_x * threads_per_block
    
    # Avoid calculating out of the array
    if idxThread >= vec_out.size:
        return
    
    # Do the calculation    
    vec_out[idxThread] = vec_a[idxThread] + vec_b[idxThread]
    

#### Launch kernel

* thread count: Set to the warp size of the GPU (or multiples of the warp size)
* block count: ceil(SizeArray/Thread_count)

This will launch more threads than elements

In [138]:
# Create data
n = 20
VecA = np.arange(n, dtype=np.float32)
VecB = np.arange(n, dtype=np.float32)
VecOut = np.empty_like(VecA)

# Launch the kernel
thread_count = myGpu.WARP_SIZE
block_count = int(math.ceil(float(n)/thread_count))

print("Threads per block", thread_count)
print("Blocks per grid", block_count)

# griddim is the number of thread-block per grid.
# blockdim is the number of threads per block
# foo[griddim, blockdim](aryA, aryB)
griddim = (block_count,1)
blockdim = (thread_count,1)

threadsperblock = myGpu.WARP_SIZE
blockspergrid = math.ceil((n + (threadsperblock - 1)) / threadsperblock)

addVecsKernel[blockspergrid,threadsperblock](VecA,VecB,VecOut)
print('VecA:\n',VecA)
print('VecB:\n',VecB)
print('VecOut:\n',VecOut)


Threads per block 32
Blocks per grid 1
VecA:
 [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.]
VecB:
 [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.]
VecOut:
 [  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.  28.
  30.  32.  34.  36.  38.]
